In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing .image import ImageDataGenerator

In [ ]:
#define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
#creating training data
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
)

In [ ]:
#creating training data with above parameters
#folder = parameters.flow_from_directory(path,ts,bs,cm,subset)
train_generator=train_datagen.flow_from_directory("/content/drive/MyDrive/Brain_Tumor_Detection-20240301T062122Z-001/Brain_Tumor_Detection/Train"
                                                  ,target_size=(IMG_SIZE,IMG_SIZE),
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='binary',
                                                 subset='training')

Found 2402 images belonging to 2 classes.


In [ ]:
#creating Validation data
train_val_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/Brain_Tumor_Detection-20240301T062122Z-001/Brain_Tumor_Detection/Train"
                                                       ,target_size=(IMG_SIZE,IMG_SIZE),
                                                 batch_size=BATCH_SIZE,
                                                 class_mode='binary',
                                                 subset='validation')

Found 600 images belonging to 2 classes.


In [ ]:
#define the model
model=keras.Sequential([
layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
layers.MaxPooling2D((2,2)),
layers.Conv2D(64,(3,3),activation='relu'),
layers.MaxPooling2D((2,2)),
layers.Conv2D(128,(3,3),activation='relu'),
layers.MaxPooling2D((2,2)),
layers.Flatten(),
layers.Dense(128,activation='relu'),
layers.Dense(1,activation='sigmoid')
])

#RNN


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
#Generating some examples sequential data
sentences = ["NTR","VENKY","COLLEGE","Recurrent Neural Networks are powerful"]

In [ ]:
#Tokenizing the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print(total_words)

9


In [ ]:
# Creating input sequences and their corresponding next words
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

[[4, 5], [4, 5, 6], [4, 5, 6, 7], [4, 5, 6, 7, 8]]

In [ ]:
# Padding sequences for consistent input size
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length,padding='pre')
print(input_sequences)

[[0 0 0 4 5]
 [0 0 4 5 6]
 [0 4 5 6 7]
 [4 5 6 7 8]]


In [ ]:
# Creating input and output data
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)


In [ ]:
# Building a simple RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_length-1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(X,y,epochs=50,verbose=2)

Epoch 1/50
1/1 - 2s - loss: 2.2545 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/50
1/1 - 0s - loss: 2.0698 - accuracy: 0.5000 - 11ms/epoch - 11ms/step
Epoch 3/50
1/1 - 0s - loss: 1.8933 - accuracy: 0.7500 - 12ms/epoch - 12ms/step
Epoch 4/50
1/1 - 0s - loss: 1.7193 - accuracy: 0.7500 - 20ms/epoch - 20ms/step
Epoch 5/50
1/1 - 0s - loss: 1.5448 - accuracy: 0.7500 - 16ms/epoch - 16ms/step
Epoch 6/50
1/1 - 0s - loss: 1.3695 - accuracy: 0.7500 - 17ms/epoch - 17ms/step
Epoch 7/50
1/1 - 0s - loss: 1.1961 - accuracy: 0.7500 - 17ms/epoch - 17ms/step
Epoch 8/50
1/1 - 0s - loss: 1.0294 - accuracy: 0.7500 - 21ms/epoch - 21ms/step
Epoch 9/50
1/1 - 0s - loss: 0.8745 - accuracy: 1.0000 - 15ms/epoch - 15ms/step
Epoch 10/50
1/1 - 0s - loss: 0.7356 - accuracy: 1.0000 - 15ms/epoch - 15ms/step
Epoch 11/50
1/1 - 0s - loss: 0.6150 - accuracy: 1.0000 - 19ms/epoch - 19ms/step
Epoch 12/50
1/1 - 0s - loss: 0.5136 - accuracy: 1.0000 - 21ms/epoch - 21ms/step
Epoch 13/50
1/1 - 0s - loss: 0.4314 - accuracy: 1

In [ ]:
#Training the model
# Generating text using the trained model
seed_text = input("Enter the starting word: ")
next_words = int(input("Enter how many words to predict: "))

for _ in range(next_words):
    tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
    tokenized_seed = pad_sequences([tokenized_seed], maxlen=max_sequence_length-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(tokenized_seed), axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    seed_text += " " + predicted_word

print(seed_text)